# 網路的基礎知識

### 網絡的基本概念

<img src="img/15.png">

### URL

In [ ]:
Uniform Resource Locator（統一資源定位符）
網路中每一個資源都對應唯一的地址—— URL

### IP地址

### IP協議

### 子網掩碼

### 網關(閘道器) Gateway

### 路由器

### DNS服務器

### 端口號

# 網路的常用命令

### IP地址的查看

### 網路的連通性測試

http://www.ruanyifeng.com/blog/2019/09/curl-reference.html 

# 容器網路的常見問題

### 容器為什麼能獲得 ip 地址

In [ ]:
docker container run -d --rm --name box1 busybox /bin/sh -c "while true; do sleep 3600; done"

In [ ]:
docker container exec -it box1 sh
ip addr 

### 為什麼宿主機可以 ping 通容器 ip

In [ ]:
ping 127.17.0.2

### 為什麼容器之間的 IP 是互通的

In [ ]:
docker container run -d --rm --name box2 busybox /bin/sh -c "while true; do sleep 3600; done"

In [ ]:
docker container exec -it box2 sh
ip a
ping 127.17.0.2

### 為什麼容器能夠 ping 通外網

In [ ]:
docker container exec -it box1 sh
$ ping www.yahoo.com.tw

### 容器端口轉發是怎麼回事

In [ ]:
docker container run -d --name web1 nginx
docker container inspect web1

In [ ]:
docker container exec -it box1 sh
$ ping 127.17.0.4
$ wget 127.17.0.4:80

In [ ]:
$ ping 127.17.0.4
$ curl 127.17.0.4:80

# 容器間通信的 bridge 網路

In [ ]:
首先我們先啟動兩個容器 box1、box2。

In [ ]:
docker container run -d --rm --name box1 busybox /bin/sh -c "while true; do sleep 3600; done"
docker container run -d --rm --name box2 busybox /bin/sh -c "while true; do sleep 3600; done"

In [ ]:
ip addr 
docker network ls

In [ ]:
docker network inspect bridgeId

<img src="img/8.png">

In [ ]:
brctl show

# 容器對外通信的 bridge 網路

In [ ]:
先啟動一個 busybox 的容器，在容器內不可以 ping 百度成功是什麼原因呢。要讓容器可以 ping 通外網的先決條件是本身的宿主機就能成功
連線。我們可以由 ip route 查看路由，連線是經由 eth0 出去。容器的 ip 轉成 eth0 的 ip 在發送出去的。

In [ ]:
docker container run -d --rm --name box1 busybox /bin/sh -c "while true; do sleep 3600; done"
docker container exec -it box1 ping www.baidu.com
ip route

In [ ]:
sudo iptables --list -t nat

NAT 簡介: https://www.udemy.com/course/docker-china/learn/lecture/27332504#overview

<img src="img/9.png">

# 如何使用自定義的 bridge

In [ ]:
docker container run -d --rm --name box1 busybox /bin/sh -c "while true; do sleep 3600; done"
docker container run -d --rm --name box2 busybox /bin/sh -c "while true; do sleep 3600; done"

In [ ]:
docker network create -d bridge myBridge

In [ ]:
docker network ls

In [ ]:
docker network inspect myBridge

In [ ]:
docker container run -d --rm --name box3 --network myBridge busybox /bin/sh -c "while true; do sleep 3600; done"

In [ ]:
docker container inspect box3

In [ ]:
docker network inspect myBridge

In [ ]:
docker container exec -it box3 sh
$ ping 172.17.0.2 

In [ ]:
docker network connect bridge box3

In [ ]:
docker container inspect box3

In [ ]:
docker network disconnet bridge box3
docker container inspect box3

<img src="img/10.png">

### 使用自定義 bridge 有類似 DNS 的功能

In [ ]:
docker container run -d --rm --name box1 busybox /bin/sh -c "while true; do sleep 3600; done"
docker container run -d --rm --name box2 busybox /bin/sh -c "while true; do sleep 3600; done"
docker network create -d bridge myBridge
docker container run -d --rm --name box3 --network myBridge busybox /bin/sh -c "while true; do sleep 3600; done"
docker container run -d --rm --name box4 --network myBridge busybox /bin/sh -c "while true; do sleep 3600; done"

In [ ]:
docker container exec -it box4 ping 172.18.0.2
docker container exec -it box4 ping box3
docker container exec -it box1 ping 172.17.0.3
docker container exec -it box1 ping box2

In [ ]:
docker network create -d bridge --gateway 172.200.0.1 --subnet 172.200.0.0/16 demoBridge
docker network inspect demoBridge

# 容器的端口轉發 Port Forwarding

<img src="img/11.png">

In [ ]:
docker container run -d --rm --name web -p 8080:80 nginx
docker container inspect web

In [ ]:
sudo iptables -t nat -nvxL 

# Dockerfile 的 EXPOSE 指令

# TCP 和 UDP 的差別

### TCP 和 UDP 是什麼

### TCP 和 UDP 如何運作

<img src="img/12.png">

# Docker 網路: host 模式

<img src="img/13.png">

In [ ]:
docker container run -d --network host --name h0 nginx

In [ ]:
docker container run -d --network host --name h1 nginx

# Docker 網路: none 模式

In [ ]:
docker container run -d --rm --name b0 --network none busybox /bin/sh -c "while true; do sleep 3600; done"
docker container exec -it b0 ip a

# 綜合演練: Python Flask + Redis

<img src="img/14.png">

In [ ]:
from flask import Flask
from redis import Redis
import os
import socket
import redis

app = Flask(__name__)
redis = Redis(host=os.environ.get("REDIS_HOST", "127.0.0.1"), port=6379)


@app.route("/")
def index():
    redis.incr("hits")
    return f"Hello Container World! I have been seen {redis.get('hits').decode('utf-8')} times and my hostname is {socket.gethostname()}.\n"

Dockerfile 檔:

In [ ]:
FROM python:3.9.5-slim
RUN pip install flask redis && \
    groupadd -r flask && useradd -r -g flask flask && \
    mkdir /src &&\
    chown -R flask:flask /src

USER flask
COPY app.py /src/app.py
WORKDIR /src
ENV FLASK_APP=app.py REDIS_HOST=redis FLASK_RUN_HOST=0.0.0.0
EXPOSE 5000
CMD ["flask", "run"]

In [ ]:
# prepare image
docker image pull redis
docker image build -t flask-demo .

# create network
docker network create -d bridge demo-network

# create container
docker container run -d --name redis-server --network demo-network redis
docker container run -d --network demo-network --name flask-demo --env REDIS_HOST=redis-server -p 5000:5000 flask-demo